In [29]:
%widescreen

In [103]:
import visual_behavior.database as db
import pandas as pd

query = {
        "traceback": {
        "$regex": 'OneResultExpectedError',
        "$options" :'i' # case-insensitive
    }
}
conn = db.Database('visual_behavior_data')
matching_errors = (pd.DataFrame(list(conn['sdk_validation']['error_logs'].find(query))).drop(columns='_id'))
conn.close()

def get_ophys_session_id(bsid):
    query = '''
    select ophys_session_id
    from behavior_sessions
    where behavior_sessions.id = {}
    '''.format(bsid)
    return db.lims_query(query.format(bsid))

def get_workflow_state(osid):
    if pd.isnull(osid):
        return None
    query = '''
        select workflow_state
        from ophys_experiments
        where ophys_session_id = {}
    '''.format(osid)
    ans = db.lims_query(query)
    if isinstance(ans, pd.DataFrame):
        # sessions with multiple experiments can have multiple workflow states
        if (ans['workflow_state'] == 'passed').any():
            # return passed if any are passed
            return 'passed'
        else:
            # otherwise, join the various states together with a "+"
            return '+'.join(np.sort(ans['workflow_state'].unique()))
    else:
        return ans

matching_errors['ophys_session_id'] = matching_errors['behavior_session_id'].map(lambda bsid: get_ophys_session_id(bsid))
matching_errors['workflow_state'] = matching_errors['ophys_session_id'].map(lambda osid: get_workflow_state(osid))

def get_project_code_and_container(osid):
    if pd.notnull(osid):
        ans = db.lims_query(
            '''
            select projects.code as project_code, oe.experiment_container_id
            from ophys_sessions os
            join ophys_experiments oe on oe.ophys_session_id = os.id
            join projects on os.project_id = projects.id
            where os.id = {}
            '''.format(osid)
        )
        return ans.iloc[0]
    else:
        return pd.Series({'project_code':None, 'experiment_container_id':None})

matching_errors['experiment_container_id'] = None
matching_errors['project_code'] = None
matching_errors[['project_code','experiment_container_id']] = matching_errors['ophys_session_id'].apply(get_project_code_and_container)


In [105]:
len(matching_errors.drop_duplicates('behavior_session_id'))

493

In [106]:
matching_errors.columns

Index(['timestamp', 'sdk_version', 'python_version', 'platform',
       'behavior_session_id', 'failed_attribute', 'error_class', 'traceback',
       'ophys_session_id', 'workflow_state', 'experiment_container_id',
       'project_code'],
      dtype='object')

In [107]:
matching_errors.drop_duplicates('behavior_session_id')[['project_code','behavior_session_id','ophys_session_id','workflow_state']]

,project_code,behavior_session_id,ophys_session_id,workflow_state
0,None,932932421,NaN,None
1,VisualBehavior,846710859,846605051.0,passed
3,VisualBehaviorIntegrationTest,768638583,768434190.0,passed
4,VisualBehaviorIntegrationTest,805639787,805419021.0,processing
8,VisBIntTestDatacube,768896439,758211190.0,failed
...,...,...,...,...
1086,None,988143231,NaN,None
1087,None,982739847,NaN,None
1093,None,1010664993,NaN,None
1094,None,1010732747,NaN,None


In [119]:
projects = [
    'VisualBehavior',
    'VisualBehaviorMultiscope',
    'VisualBehaviorMultiscope4areasx2d',
]
valid_states = [
    'qc',
    'processing',
]
ophys_subset = (
    matching_errors[pd.notnull(matching_errors['ophys_session_id'])]
    .drop_duplicates('behavior_session_id')
    .query('project_code in @projects and workflow_state in @valid_states')
)
weird_ones = ophys_subset[['project_code','behavior_session_id','ophys_session_id','workflow_state']].sort_values(by=['project_code','workflow_state']).copy()
weird_ones['ophys_session_id'] = weird_ones['ophys_session_id'].astype(int)
list(weird_ones['ophys_session_id'].values)

[900465707,
 897687788,
 896467472,
 896259731,
 1006423043,
 852023722,
 855898523,
 858899525,
 857759922,
 859459978,
 875293691,
 873248441,
 882417489,
 898849158,
 856337979,
 884545175,
 908810528,
 857075165,
 867649640,
 868621477,
 1000327470]

In [112]:
ophys_subset.groupby('project_code')['workflow_state'].value_counts()

project_code                       workflow_state
DevelopmentMultiscope4areasx2d     created+qc         1
                                   processing         1
                                   qc                 1
MesoscopeDevelopment               processing         8
                                   qc                 5
                                   created            2
                                   passed             2
VisBIntTestDatacube                failed            20
                                   passed            15
                                   processing         3
VisBehNeuroModAx                   qc                 3
                                   created+qc         1
VisBehViralDev                     qc                 4
VisualBehavior                     failed            93
                                   passed            27
                                   qc                15
                                   processing         

In [38]:
db.lims_query(
    '''
    select *
    from ophys_experiments
    where ophys_session_id = 857075165
    '''
)

,id,name,storage_directory,workflow_state,ophys_session_id,targeted_structure_id,imaging_depth_id,qc_operator_id,calculated_depth,experiment_container_id,ophys_primary_image_id,ophys_z_stack_local_image_id,averaged_surface_image_id,averaged_depth_image_id,maximum_intensity_projection_image_id,mip_isi_offset_x,mip_isi_offset_y,movie_width,movie_height,movie_number_of_frames,raw_movie_width,raw_movie_height,raw_movie_number_of_frames,ophys_imaging_plane_group_id
0,857667592,20190425_437204_3imagesA,/allen/programs/braintv/production/visualbehav...,qc,857075165,385,511480447,None,None,None,857667597,857667601,857671727,857671737,858381185,None,None,447,512,141340,512,512,141340,None


In [59]:
db.lims_query(
    '''
    select *
    from behavior_sessions
    limit 100
    '''
)

,id,behavior_training_id,created_at,updated_at,storage_directory,temp_output_file_location,foraging_id,donor_id,ophys_session_id,equipment_id,user_id,ecephys_session_id
0,687935759,668919045.0,2018-04-23 18:15:35.253832,2018-04-23 18:15:35.553691,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/18...,None,663634017,NaN,NaN,NaN,None
1,689312186,NaN,2018-04-25 00:05:37.411042,2018-04-25 00:05:37.875611,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/ne...,402a6b63-3f02-4cc7-be17-fd2f48453610,639389516,NaN,NaN,NaN,None
2,689804140,NaN,2018-04-25 23:00:33.923614,2018-04-25 23:00:34.176702,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,bf40e3b3-9be1-4837-aff1-da23871ab362,623289718,NaN,NaN,NaN,None
3,690510721,NaN,2018-04-26 17:20:34.155121,2018-04-26 17:20:34.520136,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,9c1d1c1b-a7a0-4169-8a77-45f00d38bbed,623289718,NaN,NaN,NaN,None
4,690510729,NaN,2018-04-26 17:20:34.300325,2018-04-26 17:20:35.527182,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,df2e378e-c9af-4a74-8969-c81b7ae81de1,642238406,NaN,NaN,NaN,None
5,690919752,NaN,2018-04-26 21:50:32.805882,2018-04-26 21:50:33.151115,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,4ccc5809-2e15-481d-99db-0a25c09f4048,652074239,NaN,NaN,NaN,None
6,690919756,NaN,2018-04-26 21:50:32.925317,2018-04-26 21:50:34.002883,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,eaf5ca23-8bd7-45ff-b838-df4a9d5c7acb,644812186,NaN,NaN,NaN,None
7,691413941,NaN,2018-04-27 17:15:35.769531,2018-04-27 17:15:35.988242,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,30d8b132-59df-43ff-b3b7-cfca7a973630,623289718,NaN,NaN,NaN,None
8,691721584,NaN,2018-04-27 22:25:36.670437,2018-04-27 22:25:37.117049,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,85f7a51e-fd21-420a-8e5f-b297e216c160,644812186,NaN,NaN,NaN,None
9,691721593,NaN,2018-04-27 22:25:36.785136,2018-04-27 22:25:37.939816,/allen/programs/braintv/production/neuralcodin...,/allen/programs/braintv/production/incoming/vi...,76d20be7-8982-4c1e-aa34-a9d208604973,645145570,NaN,NaN,NaN,None


In [49]:
db.lims_query(
    '''
    select *
    from ophys_sessions os
    join ophys_experiments on ophys_experiments.ophys_session_id = os.id
    where os.id = 857075165
    '''
)

,id,name,storage_directory,workflow_state,specimen_id,isi_experiment_id,parent_session_id,vasculature_image_id,targeted_structure_id,imaging_depth_id,temp_upload_directory,date_of_acquisition,equipment_id,operator_id,project_id,reticle_image_id,ophys_z_stack_column_image_id,stimulus_name,stimulus_warp,stimulus_eye_position_x,stimulus_eye_position_y,screen_height,screen_width,screen_depth,eye_track_image_id,eye_tracking_id,foraging_id,stim_delay,visual_behavior_supercontainer_id,id,name,storage_directory,workflow_state,ophys_session_id,targeted_structure_id,imaging_depth_id,qc_operator_id,calculated_depth,experiment_container_id,ophys_primary_image_id,ophys_z_stack_local_image_id,averaged_surface_image_id,averaged_depth_image_id,maximum_intensity_projection_image_id,mip_isi_offset_x,mip_isi_offset_y,movie_width,movie_height,movie_number_of_frames,raw_movie_width,raw_movie_height,raw_movie_number_of_frames,ophys_imaging_plane_group_id
0,857075165,20190425_437204_3imagesA,/allen/programs/braintv/production/visualbehav...,uploaded,803314654,811864456,856337979,857671719,385,511480447,/projects/incoming/neuralcoding/,2019-04-25 15:58:20,523490080,51,767756009,857671710,None,OPHYS_3_images_A,None,None,None,None,None,None,None,None,e30a081d-4dd6-4549-9c91-26de400d87ab,None,None,857667592,20190425_437204_3imagesA,/allen/programs/braintv/production/visualbehav...,qc,857075165,385,511480447,None,None,None,857667597,857667601,857671727,857671737,858381185,None,None,447,512,141340,512,512,141340,None


In [92]:
def get_project_code_and_container(osid):
    if pd.notnull(osid):
        ans = db.lims_query(
            '''
            select projects.code as project_code, oe.experiment_container_id
            from ophys_sessions os
            join ophys_experiments oe on oe.ophys_session_id = os.id
            join projects on os.project_id = projects.id
            where os.id = {}
            '''.format(osid)
        )
        return ans.iloc[0]
    else:
        return pd.Series({'project_code':None, 'experiment_container_id':None})

matching_errors['experiment_container_id'] = None
matching_errors['project_code'] = None
matching_errors[['project_code','experiment_container_id']] = matching_errors['ophys_session_id'].apply(get_project_code_and_container)

matching_errors.to_csv(
    '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/one_result_expected_error_log_2020.05.04.csv',
    index=False
)

In [98]:
get_project_code_and_container(857075165)

project_code               VisualBehavior
experiment_container_id              None
Name: 0, dtype: object

In [95]:
db.lims_query('select * from ophys_sessions where id = 857075165')

,id,name,storage_directory,workflow_state,specimen_id,isi_experiment_id,parent_session_id,vasculature_image_id,targeted_structure_id,imaging_depth_id,temp_upload_directory,date_of_acquisition,equipment_id,operator_id,project_id,reticle_image_id,ophys_z_stack_column_image_id,stimulus_name,stimulus_warp,stimulus_eye_position_x,stimulus_eye_position_y,screen_height,screen_width,screen_depth,eye_track_image_id,eye_tracking_id,foraging_id,stim_delay,visual_behavior_supercontainer_id
0,857075165,20190425_437204_3imagesA,/allen/programs/braintv/production/visualbehav...,uploaded,803314654,811864456,856337979,857671719,385,511480447,/projects/incoming/neuralcoding/,2019-04-25 15:58:20,523490080,51,767756009,857671710,None,OPHYS_3_images_A,None,None,None,None,None,None,None,None,e30a081d-4dd6-4549-9c91-26de400d87ab,None,None


In [99]:
db.lims_query('select id from ophys_experiments where ophys_session_id = 857075165')

857667592

In [97]:
db.lims_query('select * from ophys_experiments_visual_behavior_experiment_containers where ophys_experiment_id = 857667592')

,visual_behavior_experiment_container_id,ophys_experiment_id,id


In [102]:
db.lims_query('''select projects.code as project_code, vbec.id AS experiment_container_id
from ophys_sessions os
join ophys_experiments oe on oe.ophys_session_id = os.id
JOIN ophys_experiments_visual_behavior_experiment_containers oevbec ON oevbec.ophys_experiment_id=oe.id
JOIN visual_behavior_experiment_containers vbec ON vbec.id=oevbec.visual_behavior_experiment_container_id
join projects on os.project_id = projects.id
where os.id = {}'''.format(857075165))

,project_code,experiment_container_id


In [84]:
ans

project_code               VisualBehavior
experiment_container_id              None
Name: 0, dtype: object

In [85]:
pd.Series({'project_code':None, 'experiment_container_id':None})

project_code               None
experiment_container_id    None
dtype: object

In [86]:
matching_errors['experiment_container_id'] = None
matching_errors['project_code'] = None


In [87]:
matching_errors[['project_code','experiment_container_id']] = matching_errors['ophys_session_id'].apply(get_project_code_and_container)

In [88]:
matching_errors.sample(5)

,timestamp,sdk_version,python_version,platform,behavior_session_id,failed_attribute,error_class,traceback,ophys_session_id,workflow_state,experiment_container_id,project_code
906,2020-04-12 13:12:52.941055,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,870559928,eye_tracking,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",870238912.0,passed,None,VisualBehavior
608,2020-04-11 13:36:25.215554,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,940562558,metadata,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",940448261.0,passed,None,VisualBehaviorMultiscope
58,2020-04-09 23:58:14.585400,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,789189147,eye_tracking,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",788921784.0,failed,None,VisualBehavior
424,2020-04-10 21:29:25.823748,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,837130542,cell_specimen_table,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",836986885.0,created,None,MesoscopeDevelopment
323,2020-04-10 07:59:26.270298,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,836414424,metadata,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",836322848.0,failed,None,VisualBehavior


In [91]:
matching_errors['experiment_container_id'].unique()

array([None], dtype=object)

In [23]:
pd.isnull(matching_errors.drop_duplicates('behavior_session_id')['ophys_session_id']).value_counts()

False    329
True     164
Name: ophys_session_id, dtype: int64

In [16]:
matching_errors.drop_duplicates('behavior_session_id')['workflow_state'].value_counts()

failed        135
passed         78
qc             59
processing     45
created        10
created+qc      2
Name: workflow_state, dtype: int64

In [10]:
matching_errors[]

False    907
True     191
Name: ophys_session_id, dtype: int64

In [25]:
list(matching_errors.query('workflow_state in ["passed","qc"]')['failed_attribute'].unique())

['eye_tracking',
 'metadata',
 'average_projection',
 'cell_specimen_table',
 'corrected_fluorescence_traces',
 'dff_traces',
 'max_projection',
 'motion_correction',
 'ophys_timestamps',
 'segmentation_mask_image']

In [22]:
matching_errors.to_csv(
    '/allen/programs/braintv/workgroups/nc-ophys/visual_behavior/one_result_expected_error_log_2020.05.01.csv',
    index=False
)

,id,behavior_training_id,created_at,updated_at,storage_directory,temp_output_file_location,foraging_id,donor_id,ophys_session_id,equipment_id,user_id,ecephys_session_id
0,932932421,None,2019-08-27 20:20:59.910548,2019-08-27 20:21:01.412034,/allen/programs/braintv/production/visualbehav...,/allen/programs/braintv/production/incoming/vi...,622038ab-7572-4abd-9b20-76eddc1ba578,892374078,None,791748375,525783454,None


In [25]:
matching_errors.query('workflow_state == "multiple"')

,timestamp,sdk_version,python_version,platform,behavior_session_id,failed_attribute,error_class,traceback,ophys_session_id,workflow_state
64,2020-04-10 00:56:41.572077,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,978234608,corrected_fluorescence_traces,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.782247e+08,multiple
65,2020-04-10 00:56:41.811937,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,978234608,dff_traces,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.782247e+08,multiple
66,2020-04-10 00:57:44.947089,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,978234608,metadata,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.782247e+08,multiple
67,2020-04-10 00:57:45.665456,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,978234608,ophys_timestamps,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.782247e+08,multiple
69,2020-04-10 01:12:47.518524,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,979974202,average_projection,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.797852e+08,multiple
70,2020-04-10 01:12:47.692862,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,979974202,cell_specimen_table,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.797852e+08,multiple
71,2020-04-10 01:12:47.905386,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,979974202,corrected_fluorescence_traces,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.797852e+08,multiple
72,2020-04-10 01:12:48.074572,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,979974202,dff_traces,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.797852e+08,multiple
73,2020-04-10 01:13:10.342125,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,979974202,max_projection,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.797852e+08,multiple
74,2020-04-10 01:13:10.773592,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,979974202,metadata,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",9.797852e+08,multiple


In [101]:
matching_errors.query('ophys_session_id == 857075165')

,timestamp,sdk_version,python_version,platform,behavior_session_id,failed_attribute,error_class,traceback,ophys_session_id,workflow_state,experiment_container_id,project_code
886,2020-04-12 13:07:31.898061,1.7.0,3.7.3,Linux-3.10.0-514.26.2.el7.x86_64-x86_64-with-c...,857257295,metadata,<class 'allensdk.OneResultExpectedError'>,"Traceback (most recent call last):\n File ""/h...",857075165.0,qc,None,VisualBehavior
